In [17]:
import pandas as pd
from sklearn.linear_model import Ridge

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")

In [3]:
stats

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
1,1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
2,2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
3,3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
4,4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14087,14087,Spencer Hawes,PF,28,MIL,54,1,14.8,2.5,5.1,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45
14088,14088,Steve Novak,PF,33,MIL,8,0,2.8,0.3,0.9,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45
14089,14089,Terrence Jones,PF,25,MIL,54,12,23.5,4.3,9.1,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45
14090,14090,Thon Maker,C,19,MIL,57,34,9.9,1.5,3.2,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45


In [4]:
del stats["Unnamed: 0"]

In [6]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          50
3P            0
3PA           0
3P%        2042
2P            0
2PA           0
2P%          84
eFG%         50
FT            0
FTA           0
FT%         462
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W             0
L             0
W/L%          0
GB            0
PS/G          0
PA/G          0
SRS           0
dtype: int64

In [10]:
stats[pd.isnull(stats["3P%"])][["Player", "3PA"]]


,Player,3PA
2,Elden Campbell,0.0
3,Irving Thomas,0.0
18,Jack Haley,0.0
20,Keith Owens,0.0
30,Benoit Benjamin,0.0
...,...,...
14061,Evan Eschmeyer,0.0
14062,Gheorghe Mureșan,0.0
14064,Jim McIlvaine,0.0
14070,Mark Hendrickson,0.0


In [11]:
stats[pd.isnull(stats["FT%"])][["Player", "FTA"]]

,Player,FTA
77,John Coker,0.0
92,Jason Sasser,0.0
103,Adrian Caldwell,0.0
119,Bruno Šundov,0.0
158,Jamal Robinson,0.0
...,...,...
13951,Mark McNamara,0.0
13979,Luke Zeller,0.0
14032,Myron Brown,0.0
14054,Malcolm Lee,0.0


In [12]:
stats = stats.fillna(0)

In [13]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

In [23]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year','W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS']

In [24]:
train = stats[stats["Year"] < 2021]

In [25]:
test = stats[stats["Year"] == 2021]

In [26]:
# from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [27]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [48]:
predictions = reg.predict(test[predictors])

In [49]:
predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)

In [50]:
predictions

,predictions
630,0.013567
631,-0.013756
632,0.002414
633,-0.004421
634,0.010734
...,...
13897,-0.012571
13898,-0.011575
13899,0.016424
13900,-0.020434


In [51]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [52]:
combination

,Player,Share,predictions
630,Aaron Gordon,0.0,0.013567
631,Austin Rivers,0.0,-0.013756
632,Bol Bol,0.0,0.002414
633,Facundo Campazzo,0.0,-0.004421
634,Greg Whittington,0.0,0.010734
...,...,...,...
13897,Patty Mills,0.0,-0.012571
13898,Quinndary Weatherspoon,0.0,-0.011575
13899,Rudy Gay,0.0,0.016424
13900,Tre Jones,0.0,-0.020434


In [53]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
641,Nikola Jokić,0.961,0.154306
8624,Joel Embiid,0.580,0.162713
3651,Stephen Curry,0.449,0.142386
9907,Giannis Antetokounmpo,0.345,0.207436
1389,Chris Paul,0.138,0.072293
10997,Luka Dončić,0.042,0.151430
7464,Damian Lillard,0.038,0.116303
3536,Julius Randle,0.020,0.088877
3531,Derrick Rose,0.010,0.033001
11358,Rudy Gobert,0.008,0.095350


In [36]:
from sklearn.metrics import mean_squared_error


In [54]:
mean_squared_error(combination["Share"], combination["predictions"])

0.002666896001382871

In [55]:
combination["Share"].value_counts()

0.000    525
0.001      3
0.961      1
0.138      1
0.042      1
0.038      1
0.020      1
0.003      1
0.008      1
0.005      1
0.580      1
0.449      1
0.010      1
0.345      1
Name: Share, dtype: int64

In [56]:
combination = combination.sort_values("Share", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0]+1))

In [57]:
combination.head(10)

,Player,Share,predictions,Rk
641,Nikola Jokić,0.961,0.154306,1
8624,Joel Embiid,0.580,0.162713,2
3651,Stephen Curry,0.449,0.142386,3
9907,Giannis Antetokounmpo,0.345,0.207436,4
1389,Chris Paul,0.138,0.072293,5
10997,Luka Dončić,0.042,0.151430,6
7464,Damian Lillard,0.038,0.116303,7
3536,Julius Randle,0.020,0.088877,8
3531,Derrick Rose,0.010,0.033001,9
11358,Rudy Gobert,0.008,0.095350,10


In [60]:
combination = combination.sort_values("predictions", ascending=False)
combination["Predicted_Rk"] = list(range(1, combination.shape[0]+1))

In [61]:
combination.head(10)

,Player,Share,predictions,Rk,Predicted_Rk
9907,Giannis Antetokounmpo,0.345,0.207436,4,1
8624,Joel Embiid,0.580,0.162713,2,2
641,Nikola Jokić,0.961,0.154306,1,3
10997,Luka Dončić,0.042,0.151430,6,4
3736,LeBron James,0.001,0.147512,15,5
3651,Stephen Curry,0.449,0.142386,3,6
4177,Kevin Durant,0.000,0.141350,531,7
4174,James Harden,0.001,0.140598,13,8
11784,Zion Williamson,0.000,0.127926,251,9
3876,Russell Westbrook,0.005,0.120227,11,10


In [62]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions,Rk,Predicted_Rk
641,Nikola Jokić,0.961,0.154306,1,3
8624,Joel Embiid,0.580,0.162713,2,2
3651,Stephen Curry,0.449,0.142386,3,6
9907,Giannis Antetokounmpo,0.345,0.207436,4,1
1389,Chris Paul,0.138,0.072293,5,33
10997,Luka Dončić,0.042,0.151430,6,4
7464,Damian Lillard,0.038,0.116303,7,12
3536,Julius Randle,0.020,0.088877,8,24
3531,Derrick Rose,0.010,0.033001,9,76
11358,Rudy Gobert,0.008,0.095350,10,19


In [68]:
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps) / len(ps)

In [69]:
find_ap(combination)

0.7636363636363636

In [70]:
years = list(range(1991, 2022))

In [72]:
aps = []
all_predctions = []
for year in years[5:]:
    train = stats[stats["Year"] < year]
    test = stats[stats["Year"] == year]
    reg.fit(train[predictors], train["Share"])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
    combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
    all_predctions.append(combination)
    aps.append(find_ap(combination))

In [73]:
sum(aps) / len(aps)

0.7112884360789578

In [75]:
def add_ranks(combination):
    combination = combination.sort_values("Share", ascending=False)
    combination["Rk"] = list(range(1, combination.shape[0]+1))
    combination = combination.sort_values("predictions", ascending=False)
    combination["Predicted_Rk"] = list(range(1, combination.shape[0]+1))
    combination["Diff"] = combination["Rk"] - combination["Predicted_Rk"]
    return combination

In [80]:
ranking = add_ranks(all_predctions[1])
ranking[ranking["Rk"] < 6].sort_values("Diff", ascending=False)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
1600,Karl Malone,0.857,0.192318,1,2,-1
10524,Michael Jordan,0.832,0.167629,2,3,-1
908,Grant Hill,0.327,0.128646,3,6,-3
4682,Tim Hardaway,0.207,0.059984,4,20,-16
8248,Glen Rice,0.117,0.033110,5,53,-48


In [81]:
def backtest(stats, model, year, predictors):
    aps = []
    all_predctions = []
    for year in years[5:]:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]
        model.fit(train[predictors], train["Share"])
        predictions = reg.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combination = add_ranks(combination)
        all_predctions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps) / len(aps), aps, pd.concat(all_predctions)
    

In [82]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [83]:
mean_ap

0.7112884360789578

In [87]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)

,0,1
13,0.070001,eFG%
18,0.035041,DRB
29,0.027125,W/L%
17,0.021610,ORB
10,0.016945,2P
21,0.011635,STL
15,0.011351,FTA
22,0.011234,BLK
20,0.007455,AST
25,0.005894,PTS


In [90]:
stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year" ]].groupby("Year").apply(lambda x: x/x.mean())

In [91]:
stat_ratios

,PTS,AST,STL,BLK,3P,Year
0,1.013334,0.420714,0.961127,0.673469,0.508587,1.0
1,1.614653,1.028412,1.647646,0.673469,4.577279,1.0
2,0.311795,0.093492,0.274608,1.571429,0.000000,1.0
3,0.200440,0.186984,0.274608,0.000000,0.000000,1.0
4,2.383005,1.636110,1.784950,0.897959,1.525760,1.0
...,...,...,...,...,...,...
14087,0.735752,0.819562,0.479763,1.528302,0.650951,1.0
14088,0.071202,0.000000,0.000000,0.000000,0.130190,1.0
14089,1.281633,0.601012,1.119447,2.547170,0.520761,1.0
14090,0.474679,0.218550,0.319842,1.273585,0.650951,1.0


In [92]:
stats[["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

In [93]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,W/L%,GB,PS/G,PA/G,SRS,PTS_T,AST_R,STL_R,BLK_R,3P_R
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,0.707,5.0,106.3,99.6,6.73,1.013334,0.420714,0.961127,0.673469,0.508587
1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,0.477,...,0.707,5.0,106.3,99.6,6.73,1.614653,1.028412,1.647646,0.673469,4.577279
2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,0.455,...,0.707,5.0,106.3,99.6,6.73,0.311795,0.093492,0.274608,1.571429,0.000000
3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,0.340,...,0.707,5.0,106.3,99.6,6.73,0.200440,0.186984,0.274608,0.000000,0.000000
4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,0.492,...,0.707,5.0,106.3,99.6,6.73,2.383005,1.636110,1.784950,0.897959,1.525760


In [94]:
predictors += ["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [95]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [96]:
mean_ap

0.7208380973034985

In [97]:
stats["NPos"] = stats["Pos"].astype("category").cat.codes

In [98]:
stats["NTm"] = stats["Tm"].astype("category").cat.codes

In [99]:
stats["Pos"].unique()

array(['PF', 'SG', 'SF', 'PG', 'C', 'PG-SG', 'PF-SF', 'SG-PG', 'PF-C',
       'SG-SF', 'SF-PF', 'SF-SG', 'C-PF', 'SG-PF', 'PG-SF', 'SF-C'],
      dtype=object)

In [100]:
stats["NTm"].value_counts()

7     499
27    495
8     493
14    493
11    492
17    491
12    490
0     485
9     480
31    479
1     479
26    476
15    475
18    475
13    474
5     473
24    468
19    466
30    465
34    464
28    463
29    463
10    461
33    421
36    387
20    343
16    331
32    263
25    214
4     177
2     165
3     157
23    145
21    143
37    114
6     113
35     88
22     32
Name: NTm, dtype: int64

In [101]:
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)

mean_ap, aps, all_predictions = backtest(stats, rf, years[28:], predictors)


In [102]:
mean_ap

0.7456311092998757

In [103]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[28:], predictors)


In [104]:
mean_ap

0.7208380973034985